<a href="https://colab.research.google.com/github/miguff/Self-Driving/blob/master/Train_custom_object_detector_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive

/content/gdrive


In [ ]:
cd MyDrive/ 

/content/gdrive/MyDrive


In [ ]:
cd Machine_Learning/

/content/gdrive/MyDrive/Machine_Learning


In [ ]:
import os

#Config.py
Alapvetően, ha a gépen csinálnám és nem a felhőben akkor ez lenne a config.py file,és amikor a BASE_PATH-ra és társaira hivatkozok csak simán ami ebben van akkor, igazából a config.BASE_PATH-ra kéne, illetve az elején be kéne importálni a config-ot is

In [ ]:
#Beaállítjuk az alap mappákat, a képeket és a hozzájuk tartozó annotációkat
BASE_PATH = "/content/gdrive/MyDrive/Machine_Learning/dataset"
IMAGES_PATH = os.path.sep.join([BASE_PATH, 'images'])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, 'annotations'])


#Beallítjuk a kimeneti értékeket is, hogy hova megy a model, a tesztek stb.
BASE_OUTPUT = "//content/gdrive/MyDrive/Machine_Learning/output"
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.h5"])
LB_PATH = os.path.sep.join([BASE_OUTPUT, 'lb.pickle'])
PLOTS_PATH = os.path.sep.join([BASE_OUTPUT, "plots"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])


#Állítsuk be az alap paramétereket a tanításhoz
INIT_LR = 1e-4
NUM_EPOCHS = 40
BATCH_SIZE = 32

In [ ]:
from tensorflow.keras.applications import VGG16 #Ez a CNN architektúra amit használni és módosítani fodgunk
from tensorflow.keras.layers import Flatten #A következők adott szintek a neurális hálózatban
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split #Ennek a segítségével kell kettévenni azokat a képeket amik a trainingre mennek és azokat amik testre
from imutils import paths
import matplotlib.pyplot as plt #Grafikun ábrázolás
import numpy as np
import pickle
import cv2
import os

In [ ]:
data = []
labels = []
bboxes = []
imagePaths = []

In [ ]:
#Most elkezdjük megtölteni az előző 4 listát külnböző adatokkal
for csvPath in paths.list_files(ANNOTS_PATH, validExts=(".csv")): #Itt az előbb beállított annotációs helyről begyűjti azokat a fileokat amiknek a kiterjestése .csv
    rows = open(csvPath).read().strip().split('\n') #Majd végig megy a sorokon és enterenként elválasztja őket
    #print(rows)
    for row in rows: #Itt azon a listán végigmegyünk amit elpbb létrehoztunk
        row = row.split(',') #Majd amik benne vannak azokat a vesszőknél külön szedjük
        (filename, startX, startY, endX, endY, label) = row #és végül amik külön lettek szedve , azokat külön változóhoz rendeljük
        imagePath = os.path.sep.join([IMAGES_PATH, filename])

        print(imagePath)
        image = cv2.imread(imagePath) #Ezzel beolvassuk a képet
        (h, w) = image.shape[:2] #Ezzel kiszedjük, hogy mekkora a magassága a képnek és mekkora a szélessége

        #Most méretezzük a bounding box-okat a térbeli méretéhez a bemeneti képhez (ez egy 0-1 közötti értéket ad vissza)
        startX = float(startX) / w
        startY = float(startY) / h
        endX = float(endX) / w
        endY = float(endY) / h

        #Itt betöltjük a képeket, és átméretezzük őket 224*224*3-assá
        image = load_img(imagePath, target_size=(224, 224)) #Ez azért kell, hogy a most használatos hálózatba beleférjen
        image = img_to_array(image) #Majd numpy array-é tesszük


        #Itt betöltjük a listákba az adatoka
        data.append(image) #a data-ba a képet numpy array-ként
        labels.append(label) #A labels-be a hozzá tartozó labelt
        bboxes.append((startX, startY, endX, endY)) #A bboxes-ba betöltjük a bounding box átméretezett koordinátáit
        imagePaths.append(imagePath) #Az imagePaths-be pedig betöltjük a képeknek a helyét

/content/gdrive/MyDrive/Machine_Learning/dataset/images/road0.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road1.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road10.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road100.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road101.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road102.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road103.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road104.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road105.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road106.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road107.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road108.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road109.png
/content/gdrive/MyDrive/Machine_Learning/dataset/images/road11.png
/content/gdrive/MyDrive/Machine_Learning/dataset/image

In [ ]:
print(len(data), len(labels), len(bboxes), len(imagePaths))

1244 1244 1244 1244


In [ ]:
#Minden adatot numpy array-é konvertálunk, hogy egyszerűbben tudjunk velünk
#számolni, illetve méretezzük a bemeneti pixeleket, hogy 0 és 1 között legyenek

data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype = "float32")
imagePaths = np.array(imagePaths)


#one-hot encodingot csinálunk, azaz, úgy alakítjuk át, hogy mindig az egyiket preferálja csak
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

#Ha csak 2 osztály van akkor elég a kategórikus is
if len(lb.classes_) == 2:
    labels = to_categorical(labels)

#ezt az utolsó 2 lépést elvileg már megtudnánk csinálni magunktól is, szóval majd 
#Olvassuk újra könyvet meg az adatokat és megnézzük hogyan


Most elosztuk az adatokat, test és train-re

In [ ]:
#Most szétosztjuk 80-20 arányban a train - test egységeket
split = train_test_split(data, labels, bboxes, imagePaths,
                         test_size = 0.2, random_state = 42)
#Ez most szétosztotta, alapból 4 változó volt, 0,1,2,3 értékkel,
#A szétosztás után 8 változó lett, 0,1,2,3,4,5,6,7
#Ebből a 0 és 1, a data lesz szétosztva 80-20-ban, a 2,3 pedig a labels 80-20-ban, és stb
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

print("[Info] saving testing image paths ...")
f = open(TEST_PATHS, "w") #Megnyitjuk azt a file-t amit az elején létroztunk txt-t és abba írjuk bele a a testnke a helyét
f.write("\n".join(testPaths))
f.close()


[Info] saving testing image paths ...


Most kezdjük el felépíteni a modelt és elkezdeni trainelni is

In [ ]:
#Betöltjuk a VGG16 hálózatot, de kihagyjuk belőle a 
#vezető fully-connected szintet
vgg = VGG16(weights="imagenet", include_top=False,
            input_tensor = Input(shape=(224, 224, 3))) #ezzel beállítottuk neki, hogy milyen legyen a bemeneti értk

#Minden szintet megfagyasztunk, hogy ezek ne update-elődjenek a training process folyamán
vgg.trainable = False

#Itt pedig 'kiegyenesítjuk az kimenő értékeit a max-pooling kimenetét
flatten = vgg.output
flatten = Flatten()(flatten)#Ezt a Layert fogjuk továbbvinni


58900480/58889256 [==============================] - 0s 0us/step


Most csináljuk meg az új vezető szintet a hálózatnak, lesz benne külön a Bounding Box-nak is meg a labelnek is

In [ ]:
#Megépítjük a hálózatunk teljesen kapcsolt 'fejét', ez fogja megtippelni a 
# Bounding boxokat

#Itt a szintek 128, 54, 31, 4 neuronból/node-ból állnak
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid", name = "bounding_box")(bboxHead)
#Az utolsó azért 4 mert az a 4 neuron lesz felelős a 4 x,y koordinátáéert

#Itt megcsináljuk a második teljesen kapcsolt réteget, csak itt a class label-t fogja
#megmondani

softmaxHead = Dense(512, activation='relu')(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation='relu')(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(len(lb.classes_), activation='softmax', name = "class_label")(softmaxHead)

#Itt egyberakjuk a modellünket, ami bevesz egy értéket és visszaad egy bounding boxot és egy osztály címkét

model = Model(
    inputs=vgg.input,
    outputs = (bboxHead, softmaxHead)
)

Most kiválasztjuk, hogy milyen loss-funkciót akarunk és összeállítjuk a modelt

In [ ]:
#létrehozunk egy szótárat, hogy abba tároljuk el, hogy melyik layert milyen
#Loss funkcióval szeretnénk korrigálni

losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error"
}
#Megadjuk, hogy mennyire súlyozza őket, és azt szeretnénk, hogy egyenlően
#legyenek súlyozva

lossWeights = {
    "class_label" : 1.0,
    "bounding_box" : 1.0
}

#megcsináljuk az optimalizálót, ami most egy Adam optimalizáló
opt = Adam(learning_rate=INIT_LR)
model.compile(loss = losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [ ]:
#ezek lesznek a training kimenő értékei
#A trainTargets lesz a training set-ünk, itt adjon hozzá
#a bboxot és a label-eket, másiknál hasonlóan, csak azok a  test lesz
trainTargets = {
    "class_label" : trainLabels,
    "bounding_box" : trainBBoxes
}

#ezek pedig a tesztnek az értékei
testTargets = {
    "class_label" : testLabels,
    "bounding_box" : testBBoxes
}

In [ ]:
#Kezdjük el trenírozni a hálózatunkat
print("[Info] training model...")
H = model.fit(
    trainImages, trainTargets, #Megadjuk, hogy mi legyen az amit le eddz és összeasonlítja
    validation_data = (testImages, testTargets), #Itt az eddig nem látott adatbázison teszteli le, hogy mennyire sikere
    batch_size = BATCH_SIZE,
    epochs = NUM_EPOCHS,
    verbose = 1
)

print("[INFO] saving object detector model ...")
model.save(MODEL_PATH, save_format="h5")

print("[INFO] saving label binarizer...")
f = open(LB_PATH, "wb") #Ez azért kel, hogy a binarizált elemeket később tudjuk emberi olvasásra alkalmasan megmutatni
f.write(pickle.dumps(lb))
f.close()

[Info] training model...
Epoch 1/40
32/32 [==============================] - 23s 288ms/step - loss: 1.2185 - bounding_box_loss: 0.0270 - class_label_loss: 1.1915 - bounding_box_accuracy: 0.5920 - class_label_accuracy: 0.5859 - val_loss: 0.9652 - val_bounding_box_loss: 0.0239 - val_class_label_loss: 0.9413 - val_bounding_box_accuracy: 0.6586 - val_class_label_accuracy: 0.6265
Epoch 2/40
32/32 [==============================] - 6s 183ms/step - loss: 0.9090 - bounding_box_loss: 0.0184 - class_label_loss: 0.8906 - bounding_box_accuracy: 0.6784 - class_label_accuracy: 0.6754 - val_loss: 0.9301 - val_bounding_box_loss: 0.0213 - val_class_label_loss: 0.9089 - val_bounding_box_accuracy: 0.6305 - val_class_label_accuracy: 0.6345
Epoch 3/40
32/32 [==============================] - 6s 183ms/step - loss: 0.8228 - bounding_box_loss: 0.0149 - class_label_loss: 0.8079 - bounding_box_accuracy: 0.7337 - class_label_accuracy: 0.7166 - val_loss: 0.8043 - val_bounding_box_loss: 0.0211 - val_class_label_lo

predict.py

In [ ]:
from IPython.display import display, Javascript, Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import mimetypes #ez fel tud ismerni file típusokat filenevek és url-ek alapján
import imutils
import pickle
import cv2
import os

#input = "/home/pi/PycharmProjects/Self_Driving_2_0/Sign_Detection/60_speed.jpg"
#filetype = mimetypes.guess_type(input)
#print(filetype)

print("[INFO] loading object detector ... ")
model = load_model(MODEL_PATH)
lb = pickle.loads(open(LB_PATH, "rb").read())

cam = cv2.VideoCapture(1)

while True:

    ignore, frame = cam.read()
    image = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
    frame = cv2.resize(frame, (224, 224))
    #image = load_img(frame, target_size=(224, 224))
    image = img_to_array(image)/255.0
    image = np.expand_dims(image, axis=0)

    (boxPreds, labelPreds) = model.predict(image)
    (startX, startY, endX, endY) = boxPreds[0]

    i = np.argmax(labelPreds, axis=1)
    label = lb.classes_[i][0] #Azért kell ide a nulladik, hogy ne listában hanem stringként adja vissza az értéket

    LEVEL = 90
    if max(labelPreds[0])*100 > LEVEL:


    #image = cv2.imread(frame)
    #image = imutils.resize(image, width=600) #itt csak azért kell ez a sor, hogy az eredeti sor beférjen a képernyőre
        (h, w) = image.shape[1:3]



        startX = int(startX*w)
        startY = int(startY*h)
        endX = int(endX*w)
        endY = int(endY*h)
        #print(startX, startY, endX, endY)

        y = startY - 10 if startY -10 >10 else startY +10
        cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0,255,0), 2)

    cv2.imshow("Output", frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

[INFO] loading object detector ... 


error: ignored